# Code Gen

In [1]:
from agential.cog.prompts.agents.react import REACT_INSTRUCTION_MBPP
from agential.cog.prompts.benchmarks.mbpp import MBPP_FEWSHOT_EXAMPLES_REACT
from agential.cog.strategies.react.code import ReActCodeStrategy
from agential.cog.prompts.agents.react import (
    REACT_INSTRUCTION_HOTPOTQA,
)
from agential.cog.prompts.benchmarks.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_REACT
import warnings
warnings.filterwarnings('ignore')

import dotenv

dotenv.load_dotenv()

from langchain_community.chat_models.openai import ChatOpenAI

llm = ChatOpenAI()

strat = ReActCodeStrategy(llm)

In [2]:
question = "Write a python function to find the first repeated character in a given string."
tests = """assert first_repeated_char("abcabc") == "a"
assert first_repeated_char("abc") == None
assert first_repeated_char("123123") == "1\""""

In [ ]:
# out = strat.generate(
#     question=question,
#     examples=MBPP_FEWSHOT_EXAMPLES_REACT,
#     prompt=REACT_INSTRUCTION_MBPP,
#     additional_keys={"tests": tests},
# )

In [ ]:
# action_type, query = strat.generate_action(
#     question=question,
#     examples=MBPP_FEWSHOT_EXAMPLES_REACT,
#     prompt=REACT_INSTRUCTION_MBPP,
#     additional_keys={"tests": tests}
# )